In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [70]:
legendary_pokedex = requests.get("https://pokemon.fandom.com/wiki/Legendary_Pok%C3%A9mon")
soup = BeautifulSoup(legendary_pokedex.text, 'html.parser')
tables = soup.find_all("table", {'class': 'article-table'})
legendary_pokemon = set()
for table in tables[:-1]:
    rows = table.find('tbody').find_all('tr')
    for row in rows[1:]:
        row_data = row.find_all('td')
        pokemon_data = [data.text.strip() for data in row_data]
        legendary_pokemon.add(pokemon_data[2])

In [71]:
pokedex = requests.get("https://pokemondb.net/pokedex/all")
soup = BeautifulSoup(pokedex.text, 'html.parser')
table = soup.find('table', {'id': 'pokedex'})

In [72]:
titles = soup.find_all('div', {'class': 'sortwrap'})

In [73]:
table_titles = [title.text for title in titles]
type_index = table_titles.index('Type')
table_titles[type_index] = 'Type 1'
table_titles.insert(type_index + 1, 'Type 2')
table_titles.append('Legendary')

In [74]:
pokemon = pd.DataFrame(columns=table_titles)

In [75]:
rows = table.find('tbody').find_all('tr')

In [79]:
for row in rows:
    row_data = row.find_all('td')
    pokemon_data = [data.text.strip() for data in row_data]
    types = pokemon_data[2].split(' ')
    pokemon_data[2] = types[0]
    if len(types) > 1:
        pokemon_data.insert(3, types[1])
    else:
        pokemon_data.insert(3, 'None')
    if pokemon_data[1] in legendary_pokemon:
        pokemon_data.append('True')
    else:
        pokemon_data.append('False')

    pokemon.loc[len(pokemon)] = pokemon_data


In [81]:
pokemon.to_csv(r'./pokemon_data.csv', index = False)